In [31]:
import pandas as pd
import numpy as np
import string

In [8]:
from szp_funcs import print_df, load_jobs, load_pay_types, get_otv

In [79]:
jobs = load_jobs()['job_106'].unique().tolist()
pay_types = load_pay_types()['name'].unique().tolist()

In [80]:
rm = string.punctuation + ''.join(chr(char) for char in range(1, 32)) + ' '

In [81]:
jobs = [i.strip().lower().translate(str.maketrans('', '', rm)) for i in jobs]
pay_types = [i.strip().lower().translate(str.maketrans('', '', rm)) for i in pay_types]

In [82]:
nach_name = 'ДОНМ_Начисления работников_V2.xlsx'
pay_name = 'ДОНМ_Начисления работников по категориямv2.xlsx'

In [83]:
path_to_dirt = r'C:\Users\PetukhovMD\Desktop\2022_new\Июль 2022' + '\\'
path_to_print = r'C:\Users\PetukhovMD\Desktop\Должности и начисления' + '\\'

In [84]:
nach = pd.read_excel(path_to_dirt + nach_name)
pay = pd.read_excel(path_to_dirt + pay_name)

In [76]:
def to_format(row, name):
    return row[name].strip().lower().translate(str.maketrans('', '', rm))

In [58]:
nach_out = nach.iloc[:, [2, 5, 8]].dropna()
nach_out.columns = ['inn', 'fio', 'job']
nach_out['clean_job'] = nach_out.apply(lambda row: to_format(row, 'job'), axis=1)
nach_out

,inn,fio,job,clean_job
0,7734408766,Солодовникова Виктория Дмитриевна,Преподаватель,преподаватель
1,7734408766,Рубцова Ирина Ивановна,Преподаватель,преподаватель
2,7734408766,Афанасьева Надежда Николаевна,Библиотекарь,библиотекарь
3,7734408766,Чекмарева Наталья Александровна,Специалист,специалист
4,7734408766,Гуськова Ольга Валентиновна,Методист,методист
...,...,...,...,...
131207,7722069597,Тамразова Нелли Робертовна,Помощник воспитателя,помощниквоспитателя
131208,7722069597,Газова Анастасия Александровна,Специалист по интернет-маркетингу,специалистпоинтернетмаркетингу
131209,7722069597,Чатинян Зина Армановна,Воспитатель,воспитатель
131210,7722069597,Макарова Ольга Леонидовна,Воспитатель,воспитатель


In [68]:
print_df(nach_out[nach_out.clean_job.isin(jobs) == False].drop_duplicates(), path_to_print + 'bad_jobs')

In [85]:
pay_out = pay.iloc[:, [2, 4, 13]].dropna()
pay_out.columns = ['inn', 'fio', 'pay_name']
pay_out['clean_pay_name'] = pay_out.apply(lambda row: to_format(row, 'pay_name'), axis=1)
pay_out

,inn,fio,pay_name,clean_pay_name
0,7734408766,Солодовникова Виктория Дмитриевна,Доплата за заведование кабинетами (лаборатория...,доплатазазаведованиекабинетамилабораториямиотд...
1,7734408766,Солодовникова Виктория Дмитриевна,Доплата за увеличение объема работ,доплатазаувеличениеобъемаработ
2,7734408766,Солодовникова Виктория Дмитриевна,Оклад (должностной оклад),окладдолжностнойоклад
3,7734408766,Солодовникова Виктория Дмитриевна,Оклад (должностной оклад),окладдолжностнойоклад
4,7734408766,Рубцова Ирина Ивановна,Доплата за выполнение обязанностей временно от...,доплатазавыполнениеобязанностейвременноотсутст...
...,...,...,...,...
473160,7722069597,Алескеров Аласвар Поладхан оглы,Выплата за наличие квалификационной категории,выплатазаналичиеквалификационнойкатегории
473161,7722069597,Алескеров Аласвар Поладхан оглы,Оклад (должностной оклад),окладдолжностнойоклад
473162,7722069597,Алескеров Аласвар Поладхан оглы,Отпуск основной,отпускосновной
473163,7722069597,Алескеров Аласвар Поладхан оглы,Отпуск основной,отпускосновной


In [86]:
print_df(pay_out[pay_out.clean_pay_name.isin(pay_types) == False].drop_duplicates(), path_to_print + 'bad_pay')

In [90]:
job_direc = pd.read_excel(r'C:\Users\PetukhovMD\Desktop\справочники' + '\\job_direc.xlsx')

In [93]:
jobs_with_direc = job_direc['job'].unique().tolist()
job_direc_grps = {i: [] for i in jobs_with_direc}
for job, spec in [(i, j) for i,j in zip(job_direc['job'].to_list(), job_direc['spec'].to_list())]:
    job_direc_grps[job].append(spec)

In [94]:
job_direc_grps

{'Заместитель директора': ['по контролю качества образования',
  'по содержанию образования, конвергенции образовательных программ',
  'по социализации, воспитанию и безопасности обучающихся',
  'по управлению ресурсами'],
 'Заместитель начальника - начальник управления': ['в сфере ИКТ',
  'по административно-хозяйственной работе',
  'по закупочной деятельности',
  'по материально-техническому обеспечению',
  'по организации учебно-воспитательной деятельности',
  'правового сопровождения',
  'по техническому обслуживанию',
  'по финансово-экономической деятельности',
  '-'],
 'Проректор': ['по организационно-правовым и экономическим вопросам',
  'по развитию',
  'по учебной работе'],
 'Воспитатель': ['группы кратковременного пребывания',
  'группы ОВЗ',
  'группы продленного дня',
  'офицер кадетского класса',
  'семейного детского сада',
  '-'],
 'Мастер производственного обучения': ['по вождению',
  'учебно-производственной мастерской',
  '-'],
 'Учитель': ['алгебры',
  'английского 

In [95]:
from szp_funcs import path_to_data
jul = pd.read_excel(path_to_data + 'jul.xlsx')

In [101]:
working = jul[jul.job.isin(jobs_with_direc)][['inn', 'snils', 'job', 'spec']].fillna('-')

In [114]:
b = working[working.spec == '-'].drop(columns=['spec']).groupby(['inn', 'job']).count().unstack()
b.columns = [i[1] + ' без напр' for i in b.columns]
b

,Ведущий специалист без напр,Воспитатель без напр,Заведующий практикой без напр,Заместитель директора без напр,Заместитель начальника - начальник управления без напр,Заместитель начальника отдела без напр,Заместитель начальника службы без напр,Мастер производственного обучения без напр,Младший воспитатель без напр,Начальник группы без напр,Начальник отдела без напр,Начальник сектора без напр,Начальник службы без напр,Начальник управления без напр,Помощник воспитателя без напр,Специалист без напр,Учитель без напр
inn,,,,,,,,,,,,,,,,,
5003021368,1.0,325.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,11.0,NaN
5003021495,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,10.0,NaN
5003021640,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,3.0,NaN
5003096290,3.0,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,28.0,2.0,NaN
5030032182,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,9.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743940590,1.0,45.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,4.0,114.0
9705101759,4.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,1.0,NaN
9715206976,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,1.0,2.0


In [121]:
print_df(working[working.spec == '-'], r'C:\Users\PetukhovMD\Desktop\Должности и начисления' + '\\empty spec')

In [115]:
a = working.drop(columns=['spec']).groupby(['inn', 'job']).count().unstack()
a.columns = [i[1] + ' все' for i in a.columns]
a

,Ведущий специалист все,Воспитатель все,Заведующий практикой все,Заместитель директора все,Заместитель начальника - начальник управления все,Заместитель начальника отдела все,Заместитель начальника службы все,Мастер производственного обучения все,Младший воспитатель все,Начальник группы все,Начальник отдела все,Начальник сектора все,Начальник службы все,Начальник управления все,Помощник воспитателя все,Специалист все,Учитель все
inn,,,,,,,,,,,,,,,,,
5003021368,1.0,325.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,11.0,248.0
5003021495,1.0,230.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.0,10.0,310.0
5003021640,2.0,216.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,13.0,270.0
5003096290,4.0,133.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,42.0,5.0,71.0
5030032182,NaN,75.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,9.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743940590,1.0,45.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,4.0,139.0
9705101759,59.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,13.0,7.0,1.0,5.0,NaN,28.0,NaN
9715206976,2.0,65.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,23.0,142.0


In [119]:
c = pd.merge(a, b, how='left', on='inn')
c = c[sorted(c.columns)]
c.reset_index()

,inn,Ведущий специалист без напр,Ведущий специалист все,Воспитатель без напр,Воспитатель все,Заведующий практикой без напр,Заведующий практикой все,Заместитель директора без напр,Заместитель директора все,Заместитель начальника - начальник управления без напр,Заместитель начальника - начальник управления все,Заместитель начальника отдела без напр,Заместитель начальника отдела все,Заместитель начальника службы без напр,Заместитель начальника службы все,Мастер производственного обучения без напр,Мастер производственного обучения все,Младший воспитатель без напр,Младший воспитатель все,Начальник группы без напр,Начальник группы все,Начальник отдела без напр,Начальник отдела все,Начальник сектора без напр,Начальник сектора все,Начальник службы без напр,Начальник службы все,Начальник управления без напр,Начальник управления все,Помощник воспитателя без напр,Помощник воспитателя все,Специалист без напр,Специалист все,Учитель без напр,Учитель все
0,5003021368,1.0,1.0,325.0,325.0,NaN,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,59.0,11.0,11.0,NaN,248.0
1,5003021495,NaN,1.0,200.0,230.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,87.0,10.0,10.0,NaN,310.0
2,5003021640,NaN,2.0,200.0,216.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,104.0,3.0,13.0,NaN,270.0
3,5003096290,3.0,4.0,133.0,133.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,28.0,42.0,2.0,5.0,NaN,71.0
4,5030032182,NaN,NaN,75.0,75.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,22.0,9.0,9.0,NaN,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,7743940590,1.0,1.0,45.0,45.0,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,25.0,4.0,4.0,114.0,139.0
590,9705101759,4.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,7.0,NaN,1.0,NaN,5.0,NaN,NaN,1.0,28.0,NaN,NaN
591,9715206976,NaN,2.0,65.0,65.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,36.0,1.0,23.0,2.0,142.0
592,9715217689,1.0,1.0,48.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,28.0,1.0,1.0,60.0,60.0


In [122]:
print_df(c.reset_index(), r'C:\Users\PetukhovMD\Desktop\Должности и начисления' + '\\accum by job')

In [13]:
pay.columns

Index(['КодЕКИС', 'Организация', 'ОрганизацияИНН', 'СНИЛС', 'ФИО',
       'ДатаРождения', 'Должность', 'ДатаПриема', 'ДатаУвольнения',
       'Состояние', 'ВидЗанятости', 'Период', 'КатегорияРасчета',
       'ВидНачисления', 'ВсегоНачислено', 'КФО', 'КатегорияНачисления',
       'NumerRegion', 'NameBase', 'Abonent'],
      dtype='object')